In [1]:
# check if the GPU is working properly
!nvidia-smi
%rm -rf ./sample_data

Fri Oct 18 16:37:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Installing dependencies
%pip install numba -q
%pip install engineering-notation -q
%pip install opencv-contrib-python -q
%pip install argcomplete -q
%pip install dv-processing -q
%pip install GPUtil

In [3]:
# Clone SensorsINI/v2e into colab
!git clone https://github.com/SensorsINI/v2e

fatal: destination path 'v2e' already exists and is not an empty directory.


In [4]:
# install v2e
%cd /content/v2e
%pip install .

/content/v2e
Processing /content/v2e
  Preparing metadata (setup.py) ... done
  Created wheel for v2e: filename=v2e-1.5.1-py3-none-any.whl size=116981 sha256=37e628ae099416a25426663b301ceaa91677540a4ad3084daa3f6b3b90380a58
  Stored in directory: /tmp/pip-ephem-wheel-cache-7x41leru/wheels/77/5f/75/585e5c0362b6f0da630814aee1b3cce77ba29bfd5274e7fa38
Successfully built v2e
  Attempting uninstall: v2e
    Found existing installation: v2e 1.5.1
    Uninstalling v2e-1.5.1:
      Successfully uninstalled v2e-1.5.1


In [5]:
# mount with goole drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import multiprocessing
import subprocess
import GPUtil
import torch

# File paths
names_file_path = "/content/stop2.txt"
input_dir = "/content/drive/MyDrive/V2E/31-6623"
output_dir = "/content/drive/MyDrive/V2E/output"

def init_gpu():
    """Initialize GPU for PyTorch or TensorFlow usage."""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"\nUsing GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("\nUsing CPU")
    return device

def process_image_sequence(name):
    """Process the image sequence and generate event data using v2e."""
    name = name.strip()
    path = os.path.join(input_dir, name)
    output_folder = os.path.join(output_dir, name)

    # v2e command options
    overwrite = True
    unique_output_folder = True
    out_filename = "events.h5"
    davis_output = True

    skip_video_output = False
    dvs_exposure = "duration .033"
    output_mode = "dvs128"

    input_frame_rate = 12
    input_slowmotion_factor = 1

    timestamp_resolution = 0.001
    auto_timestamp_resolution = True

    # Event thresholds and noise settings
    condition = "Clean"
    thres, sigma, cutoff_hz, leak_rate_hz, shot_noise_rate_hz = (0.2, 0.03, 200, 5.18, 2.716)

    if condition == "Clean":
        thres, sigma, cutoff_hz, leak_rate_hz, shot_noise_rate_hz = (0.2, 0.02, 0, 0, 0)

    # Build the v2e command
    v2e_command = [
        "v2e", "-i", path, "-o", output_folder,
        "--overwrite" if overwrite else "",
        "--unique_output_folder", str(unique_output_folder).lower(),
        "--dvs_h5", out_filename,
        "--davis_output" if davis_output else "",
        "--no_preview",
        "--skip_video_output" if skip_video_output else f"--dvs_exposure {dvs_exposure}",
        "--input_frame_rate", str(input_frame_rate),
        "--input_slowmotion_factor", str(input_slowmotion_factor),
        "--disable_slomo",
        "--auto_timestamp_resolution", "true" if auto_timestamp_resolution else "false",
        "--pos_thres", str(thres),
        "--neg_thres", str(thres),
        "--sigma_thres", str(sigma),
        "--cutoff_hz", str(cutoff_hz),
        "--leak_rate_hz", str(leak_rate_hz),
        "--shot_noise_rate_hz", str(shot_noise_rate_hz),
        f"--{output_mode}"
    ]

    # Remove empty string arguments
    v2e_command = list(filter(None, v2e_command))

    # Run the command
    try:
        result = subprocess.run(" ".join(v2e_command), shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Success: {name}\nOutput:\n{result.stdout.decode()}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {name}:\n{e.stderr.decode()}")

def process_image_sequence_in_colab(name):
    """Process video with GPU management in Google Colab."""
    # Ensure GPU is available and select it
    gpu_id = GPUtil.getFirstAvailable()[0]
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    init_gpu()  # Initialize GPU (useful for other tasks)

    process_image_sequence(name)

# Read names from the file
with open(names_file_path, "r") as file:
    names = file.readlines()

if __name__ == "__main__":
    # Limit parallelism to 4-8 processes to avoid GPU contention
    num_cpus = 8

    with multiprocessing.Pool(num_cpus) as pool:
        pool.map(process_image_sequence_in_colab, names)



Using GPU: Tesla T4
Using GPU: Tesla T4
Using GPU: Tesla T4
Using GPU: Tesla T4
Using GPU: Tesla T4

Using GPU: Tesla T4




Using GPU: Tesla T4
Using GPU: Tesla T4


Success: 31
Output:


Using GPU: Tesla T4
Success: 876
Output:


Using GPU: Tesla T4
Success: 145
Output:


Using GPU: Tesla T4
Success: 640
Output:


Using GPU: Tesla T4
Success: 207
Output:


Using GPU: Tesla T4
Success: 754
Output:

Success: 1527
Output:


Using GPU: Tesla T4

Using GPU: Tesla T4
Success: 1070
Output:


Using GPU: Tesla T4
Success: 59
Output:


Using GPU: Tesla T4
Success: 891
Output:


Using GPU: Tesla T4
Success: 641
Output:


Using GPU: Tesla T4
Success: 577
Output:


Using GPU: Tesla T4
Success: 159
Output:


Using GPU: Tesla T4
Success: 755
Output:

Success: 1697
Output:


Using GPU: Tesla T4

Using GPU: Tesla T4
Success: 1173
Output:


Using GPU: Tesla T4
Success: 141
Output:


Using GPU: Tesla T4
Success: 925
Output:


Using GPU: Tesla T4
Success: 744
Output:


Using GPU: Tesla T4
Success: 622
